<a href="https://colab.research.google.com/github/mehrdadrashidian/CG-Ecosystem/blob/master/Asset_Crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import mysql.connector
import boto3
from io import BytesIO

# اتصال به دیتابیس MySQL
db_config = {
    'host': 'your_mysql_host',
    'user': 'your_mysql_user',
    'password': 'your_mysql_password',
    'database': 'your_database_name'
}

# اتصال به دیتابیس
db_connection = mysql.connector.connect(**db_config)
cursor = db_connection.cursor()

# تنظیمات AWS S3
s3_client = boto3.client('s3', aws_access_key_id='your_access_key', aws_secret_access_key='your_secret_key')

# نام باکت S3
s3_bucket_name = 'your_s3_bucket_name'

# تابع کرول برای بارگذاری دارایی‌ها
def crawl_and_store_assets():
    # گرفتن دارایی‌ها از جدول دارایی‌ها در MySQL
    cursor.execute("SELECT asset_id, asset_type, s3_key FROM assets")
    assets = cursor.fetchall()

    for asset in assets:
        asset_id, asset_type, s3_key = asset

        # دانلود فایل از S3
        try:
            s3_response = s3_client.get_object(Bucket=s3_bucket_name, Key=s3_key)
            file_content = s3_response['Body'].read()

            # ذخیره فایل محلی به طور موقت
            file_path = os.path.join('/tmp', asset_id)  # مسیر موقت برای ذخیره فایل
            with open(file_path, 'wb') as f:
                f.write(file_content)

            # استخراج ویژگی‌ها و ذخیره در Milvus
            try:
                tags = ["example_tag"]  # می‌توانید برچسب‌ها را از دیتابیس هم بگیرید
                store_asset(asset_id, asset_type, file_path, tags)
                print(f"Asset {asset_id} of type {asset_type} stored successfully.")
            except Exception as e:
                print(f"Error storing asset {asset_id}: {str(e)}")

        except Exception as e:
            print(f"Error downloading asset {asset_id} from S3: {str(e)}")

    cursor.close()
    db_connection.close()

# فراخوانی تابع کرول
crawl_and_store_assets()
